In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class QLearning():
    """
    Implements Q Learning Algorithm for openai-gym's FrozenLake8x8 Environment
    
    SFFF       (S: starting point, safe)
    FHFH       (F: frozen surface, safe)
    FFFH       (H: hole, fall to your doom)
    HFFG       (G: goal, where the frisbee is located)
    
    Agent starts at S and most traverse this icy field until they land on tile G!
    If they land on H or if they take 200 steps, the agent loses
    """
    
    
    def __init__(self, learning_rate, exploitation_rate, discount_rate):
        """
        Initializes hyper parameters
        
        @param learning_rate: INTEGER between 0 and 1 representing the size of a learning step
        @param exploitation_rate: INTEGER between 0 and 1 representing what fraction
            of the time do we choose to pick our current most optimal action rather than
            choose a random action. 
        @param discount_rate: INTEGER between 0 and 1 indicating our lookout discount rate
        """
        self.alpha = learning_rate
        self.epsilon = exploitation_rate
        self.beta = discount_rate
        self.env = gym.make('FrozenLake8x8-v0')
        
        # 2-D Matrix where rows are states and columns represent actions
        self.qtable = np.zeros((self.env.observation_space.n, self.env.action_space.n))
        self.actions = range(self.env.action_space.n)

        
    def policy(self, state, exploration=True):
        """
        Maps states to actions
        
        @param state: INTEGER representing agent's current state
        @return: INTEGER representing chosen action given some state
        """
        if exploration and np.random.uniform() > self.epsilon:
            return np.random.choice(self.actions)
        
        # Tie breaker for maximum reward
        max_value = max(self.qtable[state])
        max_indices = [i for i, val in enumerate(self.qtable[state]) if val == max_value]
        return np.random.choice(max_indices)
    
    
    def updateQtable(self, state, action, reward, next_state):
        """
        Updates a qtable element. Follows the formula:
        Q(s, a) = Q(s, a) + alpha * [Reward(s, a) + beta * max(Q'(s', for all a')) - Q(s, a)]
        """
        self.qtable[state][action] += self.alpha * \
            (reward + self.beta * max(self.qtable[next_state]) - self.qtable[state][action])
        
        
    def train(self, epochs):
        """
        Train RL Agent
        
        @params epochs: INTEGER representing number of training iterations
        """
        i = 1
        for epoch in range(epochs):
            if epoch/epochs > i/5:
                print("\nFinished {0}/5 epochs".format(i))
                i += 1
            self.sample(update_qtable=True)
        print("\nFinished {0}/5 epochs".format(i))
    
    
    @staticmethod
    def reward_modifier(reward, done):
        """
        Modifies reward to some extent. For example, uses inputs of
        reward and done param to determine whether the agent moved, whether they
        died, and whether they won and adjusts the reward accordingly. Without
        this function, reward will be 1 on win and 0 otherwise. We want to penalize
        excess moves, dying, and give a big reward towards winning.
        
        @param reward: INTEGER representing reward from some state, action
        @param done: BOOLEAN representing whether episode has ended
        @returns: INTEGER representing modified reward
        """
        if reward == 0 and done:
            # Died or time ran out
            reward = -200
            
        elif reward == 1:
            # Got to goal
            reward = 500
            
        else:
            # Moved
            reward = -1
            
        return reward
    
    
    def sample(self, update_qtable=False):
        """
        Undergoes one episode
        
        @param update_qtable: BOOLEAN indicating whether this episode
        will update the qtable
        """
        next_state = self.env.reset()
        
        done = False
        if not update_qtable:
            self.env.render()
            
        while not done:
            state = next_state
            action = self.policy(state, update_qtable)
            
            next_state, reward, done, info = self.env.step(action)
            
            if not update_qtable:
                self.env.render()
            else:
                reward = self.reward_modifier(reward, done)
                self.updateQtable(state, action, reward, next_state)